# Example 2 - Desigining Peptides with a Lower Hemolytic Activity

dZiner can work with different textual representation of materials, not just SMILES. In this example, I show how you can apply it design peptides with a lower hemolytic activity, given their sequence which contains the 20 essential amino acids (AA) in some order. The surrogate model in this case is a bi-directional LSTM that learns the distant-based dependency information between the AA residues within the sequence and map it to sequence's property. See [paper](https://pubs.acs.org/doi/full/10.1021/acs.jcim.2c01317) for more details on the model.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys
import dziner
import os
from langchain._api import LangChainDeprecationWarning
warnings.simplefilter("ignore", category=LangChainDeprecationWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
os.environ["ANTHROPIC_API_KEY"] = 'YOUR_ANTHROPIC_API_KEY'

## Defining Customized Agent Tools

### Setting up the hemolytic activity predictor

In [10]:
import tensorflow as tf
from langchain.agents import tool
import numpy as np

def encoder(seq, max_length=190):
    alphabet = ['A','R','N','D','C','Q','E','G','H','I', 'L','K','M','F','P','S','T','W','Y','V']
    vec = np.zeros(max_length)
    for i,l in enumerate(seq):
        vec[i] = alphabet.index(l) + 1
    return vec[np.newaxis, ...]

def counts_aa(vec):
    counts =  tf.histogram_fixed_width(vec, [0, 20], nbins=21)[1:]
    return counts/tf.reduce_sum(counts)

@tool
def predict_hemolysis(sequence):
    '''
    This tool predicts the probability of the hemolytic activity of a peptide sequence.
    The model is based on a Bidirectional LSTM from this paper:
    https://pubs.acs.org/doi/full/10.1021/acs.jcim.2c01317
    '''
    sequence = sequence.replace('\n', '')
    with open('../dziner/surrogates/peptides/hemo_model.json', 'r') as f:
        loaded_model_json = f.read()
    loaded_model = tf.keras.models.model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights('../dziner/surrogates/peptides/hemo.weights.h5')
    encoded_seq = encoder(sequence)

    hemolytic_probability =  loaded_model.predict([encoded_seq, np.array([counts_aa(xi) for xi in encoded_seq])])[0][0]
    return hemolytic_probability

### Retrieving domain-knowledge from scientific literature

We use obtain some domain-knowledge on how the presence of different amino acids can affect the hemolytic activity within the peptide sequence.

In [11]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
Embedding_model = 'text-embedding-3-large' 

RetrievalQA_prompt = """What are the design guidelines for making a peptide with a lower hemolytic activity? 
    This can be based on making changes to the amino acid residues within the sequence. Summarize answer"""

@tool
def domain_knowledge(prompt):
    '''This tool looks up design guidelines for a peptides with lower hemolytic activity by looking through research papers.
    '''
    guide_lines = []
    for m in range(2):
        text_splitter = CharacterTextSplitter(
            chunk_size=1000, chunk_overlap=50)
        paper_file =f'../data/papers/Peptides/{m}.pdf'
        pages = PyPDFLoader(paper_file).load_and_split()
        sliced_pages = text_splitter.split_documents(pages)
        faiss_vectorstore = FAISS.from_documents(sliced_pages, OpenAIEmbeddings(model=Embedding_model))
        
        llm=ChatOpenAI(
                        model_name='gpt-4o',
                        temperature=0.1,
                        )
        g = dziner.RetrievalQABypassTokenLimit(faiss_vectorstore, RetrievalQA_prompt, llm)
        guide_lines.append(g)
        # break
    return " ".join(guide_lines)

## Defining dZiner agent

We use Claude Sonnet 3.5 to drive our agent.

In [12]:
tools = [domain_knowledge, predict_hemolysis]

tool_names = [tool.name for tool in tools]  
tool_desc = [tool.description for tool in tools]

initial_sequence = "FLHSIGKAIGRLLR" 
Human_prompt = f"Make changes to {initial_sequence} so it will have a lower hemolytic activity"

input_data = {
            "input": Human_prompt,
            "prompt": RetrievalQA_prompt,
            "tools": tools,
            "tool_names": tool_names,
            "tool_desc": tool_desc
        }

### Defining custom system messages for the agent

In [13]:
FORMAT_INSTRUCTIONS = """Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:
'''
Thought: Here's your final answer:
Final Answer: [your response here]
'''

Use the exact sequebce without any "\n" or any extra "()".
"""

SUFFIX = """You should always follow these steps:
    1. Lookup and summarize design guidelines. Make sure to add citations (paper DOI) for design guidelines if you use them, in your final response.
    Your final response should also contain the source for the tools used from their summary in description in {tool_desc}.
    2. Evaluate the hemolytic activity of the initial sequence.
    3. Start making changes to the sequence based on guidelines you found in step 1. Try to use different changes at each iteration.
    You are only allowed to make changes based on the 20 essential AA in this list: ['A','R','N','D','C','Q','E','G','H','I', 'L','K','M','F','P','S','T','W','Y','V']
    4. Evaluate the hemolytic activity for the new sequence
    5. If the hemolytic activity does not change properly, revert change to the best previous sequence and try something else by
    redoing steps 3-5. Iterate until there are 3 new sequence candidates and then stop.

    Start by describing the problem: \n\nBegin!\n \n\nQuestion: {input}
Thought:{agent_scratchpad}\n"""

In [14]:
from dziner.agents import dZiner
from langchain_anthropic import ChatAnthropic

agent_model = ChatAnthropic(model="claude-3-5-sonnet-20240620", api_key=os.environ["ANTHROPIC_API_KEY"],
                      temperature=0.3, max_tokens=8192)


agent = dZiner(tools, property="hemolytic activity",
               model=agent_model, verbose=True, n_design_iterations=3, temp=0.1,
              suffix=SUFFIX, format_instructions=FORMAT_INSTRUCTIONS).agent

In [15]:
response = agent.invoke(input_data)



> Entering new AgentExecutor chain...
Thought: To address this problem, I need to first understand the design guidelines for peptides with lower hemolytic activity, then evaluate the initial sequence and make changes accordingly. Let's start by looking up the design guidelines.

Action: domain_knowledge
Action Input: design guidelines for peptides with lower hemolytic activity

Observation: To design a peptide with lower hemolytic activity, consider the following guidelines based on modifications to the amino acid residues within the sequence:

1. **Stabilize Helical Conformation**: Use analogs that stabilize the helical structure without promoting hemolysis. For example, incorporating lactam-bridged residues between specific amino acids can stabilize the helix while reducing hemolytic activity.

2. **Modify Side Chains**: Alter the side chains of amino acids to reduce interactions with red blood cell (RBC) membranes. This can involve using less hydrophobic or less positively charged